# Compound Data Curation Pipeline

**Purpose**: Extract CID and SMILES from compound names using PubChem API

**Input**: CSV file with compound names
**Output**: Clean dataset ready for SwissTarget analysis

**Note**: SwissTarget prediction will be handled separately using local Python script

## Cell 1: Setup and Configuration

In [ ]:
# Install required libraries
!pip install pandas requests

# Import libraries
import pandas as pd
import urllib.request, urllib.error, urllib.parse, json, time, csv
import os
from datetime import datetime

# --- Folder and Logging Configuration ---
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
main_folder_name = f"curation_results_{timestamp}"
os.makedirs(main_folder_name, exist_ok=True)

log_file_path = os.path.join(main_folder_name, "curation_log.txt")

def write_log(message):
    """Function to write messages to log file and print to console."""
    log_message = f"[{datetime.now().strftime('%H:%M:%S')}] {message}"
    print(log_message)
    with open(log_file_path, "a", encoding='utf-8') as f:
        f.write(log_message + "\n")

write_log("--- COMPOUND CURATION PIPELINE STARTED ---")
write_log(f"Results folder: '{main_folder_name}'")

print("✅ Setup completed successfully!")

## Cell 2: Input Data Validation

In [ ]:
# Read and validate input file
input_filename = 'compound_data.csv'

try:
    df_input = pd.read_csv(input_filename, sep=',', encoding='utf-8')
    write_log(f"✅ Successfully read input file: '{input_filename}'")
    write_log(f"   - Total compounds: {len(df_input)}")
    write_log(f"   - Columns: {list(df_input.columns)}")
    
    # Validate required columns
    if 'Name' not in df_input.columns:
        raise ValueError("Required column 'Name' not found in input file")
    
    # Preview data
    print("\n--- Input Data Preview ---")
    print(df_input.head())
    
    # Check for duplicates
    duplicates = df_input['Name'].duplicated().sum()
    if duplicates > 0:
        write_log(f"⚠️ Warning: Found {duplicates} duplicate compound names")
    
except FileNotFoundError:
    write_log(f"❌ CRITICAL: File '{input_filename}' not found. Please upload the file.")
    df_input = pd.DataFrame()
except Exception as e:
    write_log(f"❌ CRITICAL: Failed to read input file. Error: {e}")
    df_input = pd.DataFrame()

## Cell 3: PubChem API Functions

In [ ]:
def get_cid_and_smiles(compound_name):
    """
    Search for CID and SMILES using compound name variations.
    Returns (cid, smiles, status).
    """
    original_name = compound_name.strip()
    
    # Create name variations to improve search success
    alpha_name = original_name.replace('α', 'alpha').replace('β', 'beta').replace('γ', 'gamma')
    names_to_try = list(dict.fromkeys([original_name, alpha_name]))
    
    for i, name_attempt in enumerate(names_to_try):
        write_log(f"    > Searching: '{name_attempt}'")
        
        try:
            # Step 1: Get CID from compound name
            name_encoded = urllib.parse.quote(name_attempt)
            cid_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name_encoded}/cids/JSON"
            
            with urllib.request.urlopen(cid_url, timeout=20) as cid_request:
                cid_reply = cid_request.read()
                if not (cid_reply and len(cid_reply) > 0):
                    continue
                
                cid_result = json.loads(cid_reply)['IdentifierList']['CID'][0]
                write_log(f"    > Found CID: {cid_result}")
                
            # Step 2: Get SMILES from CID
            smiles_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid_result}/property/SMILES/JSON"
            
            with urllib.request.urlopen(smiles_url, timeout=20) as smiles_request:
                smiles_reply = smiles_request.read()
                if smiles_reply and len(smiles_reply) > 0:
                    smiles_result = json.loads(smiles_reply)['PropertyTable']['Properties'][0]['SMILES']
                    
                    status_result = 'Success (Original Name)' if i == 0 else 'Success (Name Variant)'
                    write_log(f"    ✅ SUCCESS: CID {cid_result}, SMILES obtained")
                    return cid_result, smiles_result, status_result
                    
        except Exception as e:
            write_log(f"    > Failed for '{name_attempt}': {str(e)[:50]}...")
            continue
    
    write_log(f"    ❌ All attempts failed for '{original_name}'")
    return None, None, 'Failed'

print("✅ PubChem API functions defined successfully!")

## Cell 4: Data Processing and CID/SMILES Extraction

In [ ]:
# Main processing logic
if len(df_input) == 0:
    write_log("❌ No input data available. Please check the input file.")
else:
    # Prepare output filename
    output_filename = os.path.join(main_folder_name, 'compounds_with_cid_smiles.csv')
    
    # Initialize counters
    success_count = 0
    failed_count = 0
    failed_compounds = []
    
    write_log(f"🚀 Starting CID and SMILES extraction for {len(df_input)} compounds...")
    
    # Process each compound
    results = []
    
    for index, row in df_input.iterrows():
        compound_name = str(row['Name']).strip()
        write_log(f"\n[{index + 1}/{len(df_input)}] Processing: '{compound_name}'")
        
        # Get CID and SMILES
        cid, smiles, status = get_cid_and_smiles(compound_name)
        
        # Update counters
        if 'Success' in status:
            success_count += 1
        else:
            failed_count += 1
            failed_compounds.append(compound_name)
        
        # Prepare result row
        result_row = {
            'Name': compound_name,
            'PubChem_CID': cid,
            'Smiles': smiles,
            'Status': status
        }
        
        # Add original columns if they exist
        for col in df_input.columns:
            if col != 'Name':
                result_row[col] = row[col]
        
        results.append(result_row)
        
        # Be respectful to PubChem servers
        time.sleep(0.5)
    
    # Create results DataFrame
    df_results = pd.DataFrame(results)
    
    # Save results
    df_results.to_csv(output_filename, index=False, encoding='utf-8')
    
    # Print summary
    write_log("\n" + "="*50)
    write_log("✨ PROCESSING COMPLETED ✨")
    write_log(f"  - Total compounds processed: {len(df_input)}")
    write_log(f"  - ✅ Successful: {success_count}")
    write_log(f"  - ❌ Failed: {failed_count}")
    write_log(f"  - Success rate: {(success_count/len(df_input)*100):.1f}%")
    
    if failed_compounds:
        write_log("\n--- Failed Compounds ---")
        for name in failed_compounds:
            write_log(f"  - {name}")
    
    write_log(f"\n💾 Results saved to: '{output_filename}'")
    write_log("="*50)
    
    # Preview results
    print("\n--- Results Preview ---")
    print(df_results.head())

## Cell 5: Data Quality Control and Cleaning

In [ ]:
# Quality control and data cleaning
if 'df_results' in locals() and len(df_results) > 0:
    write_log("\n--- QUALITY CONTROL PHASE ---")
    
    # Filter successful compounds
    success_mask = df_results['Status'].str.contains('Success', na=False)
    df_clean = df_results[success_mask].copy()
    
    # Additional validation
    initial_count = len(df_clean)
    
    # Remove rows with missing SMILES
    df_clean = df_clean.dropna(subset=['Smiles'])
    
    # Remove rows with empty SMILES
    df_clean = df_clean[df_clean['Smiles'].str.strip() != '']
    
    final_count = len(df_clean)
    removed_count = initial_count - final_count
    
    write_log(f"Quality control results:")
    write_log(f"  - Initial successful compounds: {initial_count}")
    write_log(f"  - Removed due to invalid SMILES: {removed_count}")
    write_log(f"  - Final clean compounds: {final_count}")
    
    # Save clean dataset
    clean_filename = os.path.join(main_folder_name, 'data_compounds_final_full.csv')
    df_clean.to_csv(clean_filename, index=False, encoding='utf-8')
    
    write_log(f"\n✅ Clean dataset saved to: '{clean_filename}'")
    write_log("   This file is ready for SwissTarget analysis!")
    
    # Final statistics
    write_log("\n--- FINAL STATISTICS ---")
    write_log(f"  - Original compounds: {len(df_input)}")
    write_log(f"  - Successfully processed: {success_count}")
    write_log(f"  - Ready for SwissTarget: {final_count}")
    write_log(f"  - Overall success rate: {(final_count/len(df_input)*100):.1f}%")
    
    # Preview clean data
    print("\n--- Clean Dataset Preview ---")
    print(df_clean[['Name', 'PubChem_CID', 'Smiles', 'Status']].head(10))
    
    print(f"\n🎯 Next Step: Use the file '{clean_filename}' with your SwissTarget Python script!")
    
else:
    write_log("❌ No results available for quality control. Please run the processing step first.")

## Cell 6: Download Results (Optional)

In [ ]:
# Optional: Download results in Google Colab
try:
    from google.colab import files
    
    # Check if files exist
    if 'clean_filename' in locals():
        print("Downloading clean dataset...")
        files.download(clean_filename)
        
        print("Downloading processing log...")
        files.download(log_file_path)
        
        print("✅ Files downloaded successfully!")
    else:
        print("❌ No clean dataset available for download.")
        
except ImportError:
    print("ℹ️ Not running in Google Colab. Files are saved locally.")
except Exception as e:
    print(f"⚠️ Download failed: {e}")

print("\n🎉 Compound curation pipeline completed!")
print("   You can now use the cleaned dataset with your SwissTarget analysis script.")